In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import keras.layers as layers
import keras.optimizers as optimizers
from keras.models import Model, load_model
from keras.utils import to_categorical
from keras.callbacks import LambdaCallback, ModelCheckpoint, ReduceLROnPlateau
import seaborn as sns
from PIL import Image
from skimage.transform import resize

import threading, random, os

# Data Generator

원본 이미지와 라벨 데이터를 가져오고, 이 이미지와 라벨을 패널로 나누어 학습 데이터와 라벨을 생성하는 커스텀 생성기를 만드는 부분입니다.

## Load Dataset

원본 이미지와 라벨 데이터를 가져온다. 코드를 실행하기 전에 코드와 같은 폴더에 데이터셋을 다운받는다.   

https://www.kaggle.com/kairess/find-waldo

In [4]:
imgs = np.load('dataset/imgs_uint8.npy').astype(np.float32) / 255.
labels = np.load('dataset/labels_uint8.npy').astype(np.float32) / 255.
waldo_sub_imgs = np.load('dataset/waldo_sub_imgs_uint8.npy') / 255.
waldo_sub_labels = np.load('dataset/waldo_sub_labels_uint8.npy') / 255.

ValueError: Object arrays cannot be loaded when allow_pickle=False

원본 코드 작성시와 numpy 버전 차이가 있어 waldo_sub_imgs와 waldo_sub_labels를 로드할 때 오류가 발생한다. allow_pickle 파라메터의 기본값이 True에서 False로 바뀌어 발생하는 문제로 ```allow_pickle=True```를 코드에 추가해주면 해결된다.

pickle은 파이썬에서 제공하는 모듈로 자료형의 변화 없이 데이터를 저장하고 불러올 수 있게해준다. 다만 악의적으로 제작된 데이터를 불러올 경우 임의로 코드를 실행하는 문제가 있을 수 있기 때문에 보안상의 문제로 기본값이 변경된 것으로 보인다.

In [13]:
imgs = np.load('dataset/imgs_uint8.npy').astype(np.float32) / 255.
labels = np.load('dataset/labels_uint8.npy').astype(np.float32) / 255.
waldo_sub_imgs = np.load('dataset/waldo_sub_imgs_uint8.npy', allow_pickle=True) / 255.
waldo_sub_labels = np.load('dataset/waldo_sub_labels_uint8.npy', allow_pickle=True) / 255.

In [5]:
print(imgs.shape, labels.shape)

(18, 1760, 2800, 3) (18, 1760, 2800)


먼저 imgs와 labels의 형태를 확인한다.   
imgs는 1760*2800 크기의 이미지 18장으로 RGB 데이터값이 입력된 numpy배열이다.
labels는 똑같은 이미지에서 Waldo가 있는 부분을 255, 아닌 부분을 0으로 표시한 numpy배열이다.

데이터를 학습에 적합한 형태로 (0과 1사이의 값으로) 바꾸기 위해 255로 나눠준다.
